# Classes: creating you own data type

## 1. Types and instances
We have thus far seen about a dozen different data types: we started this course exploring the primitive types, including `strings`, `floats`, `integers`, `booleans` and the special `None` type. We then introduced some container-like types, including `lists`, `tuples`, `sets` and `dictionaries`. Finally, we used a few imported types such as `date`, `time` and `datetime` types. A **type** - also known as a **class** - is an *abstract* concept that defines a group of values and how each instance of that group should behave and interact with each other.

For example: the `boolean` type defines two values: `True` and `False`, intended to represent the truthfulness of a given statement. 

In comparison, the number 3, the string `'David is awesome'` and `{'France':'Paris'}` are **instances** of the integer, the string and the dictionary types. These are just one of many possible value of each of the above types.  

### 1.1. Checking for type
We can check whether a value is of a given type using the `isinstance` built-in function

In [1]:
print(isinstance(True, bool))
print(isinstance(1, int))
print(isinstance(2.5, float))
print(isinstance('Pure happiness', str))
print(isinstance([2,4,6,8,10], list))
#and so forth...

True
True
True
True
True


In [2]:
#The isinstance function can take a tuple of types too
print(isinstance(3.1415, (int, float)))

#this is equivalent to 
print(isinstance(3.1415, int) or isinstance(3.1415, float))

True
True


## 2. Creating your own data types
Python also allows us to create our own data types, also known as `classes`. But first, let's see using a trivial example why using a user-defined custom class can help us write robust and elegant solutions.   

### 2.1. Code is about modelization...
Imagine you wanted to model a bank account and create a specific type for that purpose. For simplicity, assume a bank account has a number (e.g. IBAN), a currency (e.g. USD) and records the history of transactions, including cash withdrawals and deposits and other non-cash transactions (payments).

In [3]:
#we define a new class using the class keyword
#think of a class as a template
#the convention is to capitalize classes
class Account:
    #the __init__ method is called when you create a new instance of Account
    #the self keyword refers to the instance being created
    def __init__(self, number, currency, balance=0):
        self.number       = number
        self.currency     = currency
        self.transactions = []
        self.balance      = balance

In [4]:
#now create a new account instance
#like functions, you create a new class instance by calling it
checking = Account(number="#12345678", currency="USD")

#checking is an instance of Account
#just like 3 is an instance of int
#and has tree properties
print("Account number:", checking.number)
print("Account currency:", checking.currency)
print("Account transactions:", checking.transactions)
print("Account balance:", checking.balance)

Account number: #12345678
Account currency: USD
Account transactions: []
Account balance: 0


In [5]:
#create another instance of account
savings = Account("#987654321", "USD", balance=100)

#savings is also an instance of Account
#just like 10 is also an instance of int
#and has tree properties
print("Account number:", savings.number)
print("Account currency:", savings.currency)
print("Account transactions:", savings.transactions)
print("Account balance:", savings.balance)

Account number: #987654321
Account currency: USD
Account transactions: []
Account balance: 100


### 2.2.  ... and classes are about encapsulation

Let's now record a transaction to our checking account. For simplicity, a transaction will be a dictionary composed of: 
1. an amount: a positive number
2. a type: a string representing the type of transaction, where `DEBIT` is for outgoing payments and `CREDIT` is for incoming payments
3. a brief description.

In [6]:
#for our checking account
t1 = {"amount":10000, "type":'CREDIT', "description":"Initial deposit"}
checking.transactions.append(t1)

In [7]:
#Can you spot the problem? 
print("Transactions:", checking.transactions)
print("Balance (USD):", checking.balance) #this was not changed!

Transactions: [{'amount': 10000, 'type': 'CREDIT', 'description': 'Initial deposit'}]
Balance (USD): 0


In [8]:
#Let's therefore rewrite the Account class with a method to record a transaction
#This method will add the transaction to the history AND amend the balance

class Account:
    def __init__(self, number, currency, balance=0):
        self.number       = number
        self.currency     = currency
        self.transactions = []
        self.balance      = balance
        
    def record(self, transaction):
        self.transactions.append(transaction)
        if transaction["type"] == "CREDIT":
            self.balance += transaction["amount"]
        else: 
            self.balance -= transaction["amount"]
            

In [9]:
#Let's re-create our checking account
checking = Account(number="#12345678", currency="USD")
checking.record({"amount":10000, "type":'CREDIT', "description":"Initial deposit"})

#It's much better now! 
print("Transactions:", checking.transactions)
print("Balance (USD):", checking.balance) #this was not changed!

Transactions: [{'amount': 10000, 'type': 'CREDIT', 'description': 'Initial deposit'}]
Balance (USD): 10000


### 2.3. Encapsulated-validation
What happens now if our transaction dictionary is not well forme? 

In [10]:
#this transaction has a negative amount, no type and no description
checking.record({"amount":-3000, "type":None})

print("Transactions:", checking.transactions)
print("Balance (USD):", checking.balance)    
#Our account history is corrupted!

Transactions: [{'amount': 10000, 'type': 'CREDIT', 'description': 'Initial deposit'}, {'amount': -3000, 'type': None}]
Balance (USD): 13000


In [11]:
#Let's rewrite the class with a few validations
class Account:
    def __init__(self, number, currency, balance=0):
        #Let's make sure the initial balance is a number
        if not isinstance(balance, (int, float)): 
            raise ValueError("The initial balance should be a number")
            
        self.number       = number
        self.currency     = currency
        self.transactions = []
        self.balance      = balance
        
    def record(self, transaction):
        #let's make sure our transaction is a dictionary
        if not isinstance(transaction, dict):
            raise ValueError("The transaction should be a dictionary")
            
        #let's make sure each transaction has the three expected fields
        if not all([field in transaction for field in ["amount", "type","description"]]):
            raise ValueError("Each transaction should have an amount, a type and a description")
            
        #let's make sure type is either 'CREDIT' or 'DEBIT'
        if not transaction['type'] in ['CREDIT', 'DEBIT']:
            raise ValueError('The transaction type should be one of CREDIT or DEBIT')
            
        #let's make sure the amount is positive number (float or integer)
        if not isinstance(transaction["amount"], (int, float)) or transaction["amount"] < 0: 
            raise ValueError("The transaction amount should be a positive number")
            
        #All good?
        #Let's record the transaction and amend the balance
        self.transactions.append(transaction)
        if transaction["type"] == "CREDIT":
            self.balance += transaction["amount"]
        else: 
            self.balance -= transaction["amount"]

In [12]:
checking = Account(number="#12345678", currency="USD")
checking.record({"amount":10000, "type":'CREDIT', "description":"Initial deposit"})

print("Transactions:", checking.transactions)
print("Balance (USD):", checking.balance)

#the below will throw an error, so let's catch it and print it
try: 
    checking.record({"amount":-3000, "type":None})
except ValueError as error:
    print("Error:", error)

Transactions: [{'amount': 10000, 'type': 'CREDIT', 'description': 'Initial deposit'}]
Balance (USD): 10000
Error: Each transaction should have an amount, a type and a description


### 2.4. Creating an interface: adding more methods

In [13]:
#Let's add a few methods
#Let's also comment our code
class Account:
    def __init__(self, number, currency, balance=0):
        """
        Bank account model
        
        Arguments
        ---------
        number   : the bank account number (e.g. IBAN)
        currency : the three-letter currency code of the account (e.g. USD) 
        balance  : the initial bank account balance
        
        Properties
        --------- 
        number       : the bank account number (e.g. IBAN)
        currency     : the three-letter currency code of the account (e.g. USD) 
        balance      : the initial bank account balance
        transactions : a list of transactions
        """
        #Let's make sure the initial balance is a number
        if not isinstance(balance, (int, float)): 
            raise ValueError("The initial balance should be a number")
            
        self.number       = number
        self.currency     = currency
        self.transactions = []
        self.balance      = balance
        
    def record(self, transaction):
        """
        Method to record a new transaction. 
        
        Arguments
        ---------
        transaction  : a dictionary representing the transaction
        
        Remarks
        ---------
        The method will check the transaction is well formed
        """
        #let's make sure our transaction is a dictionary
        if not isinstance(transaction, dict):
            raise ValueError("The transaction should be a dictionary")
            
        #let's make sure each transaction has the three expected fields
        if not all([field in transaction for field in ["amount", "type","description"]]):
            raise ValueError("Each transaction should have an amount, a type and a description")
            
        #let's make sure type is either 'CREDIT' or 'DEBIT'
        if not transaction['type'] in ['CREDIT', 'DEBIT']:
            raise ValueError('The transaction type should be one of CREDIT or DEBIT')
            
        #let's make sure the amount is positive number (float or integer)
        if not isinstance(transaction["amount"], (int, float)) or transaction["amount"] < 0: 
            raise ValueError("The transaction amount should be a positive number")
            
        #All good?
        #Let's record the transaction and amend the balance
        self.transactions.append(transaction)
        if transaction["type"] == "CREDIT":
            self.balance += transaction["amount"]
        else: 
            self.balance -= transaction["amount"]
            
    def debits(self):
        """
        Method to return a list of all the debit transactions made to the account
        """
        return [transaction for transaction in self.transactions if transaction["type"] == 'DEBIT']
    
    def credits(self):
        """
        Method to return a list of all the credit transactions made to the account
        """
        return [transaction for transaction in self.transactions if transaction["type"] == 'CREDIT']
    
    def deposit(self, amount):
        """
        Convenience method to add a new cash deposit
        """
        self.record({'amount':amount, 'type':'CREDIT', 'description':'Cash deposit'})
        return self
        
    def withdraw(self, amount):
        """
        Convenience method to add a new cash deposit
        """
        self.record({'amount':amount, 'type':'DEBIT', 'description':'Cash withdrawal'})
        return self
    
    def send(self, recipient, amount):
        """
        Method to send money from this account to another recipient
        Debits this account and credits the other account
        
        Arguments
        ---------
        recipient : other account to send the payment to
        amount    : amount to be sent
        
        """
        #let's make sure the recipient is an account type
        if not isinstance(recipient, Account):
            raise ValueError("The recipient should an Account, {} given".format(type(recipient)))
            
        #let's make sure the two accounts are in the same currency
        if self.currency != recipient.currency: 
            raise ValueError("The other account should be in the same currency")
            
        self.record({"amount":amount, "type":'DEBIT', "description":"Payment to {}".format(recipient.number)})
        recipient.record({"amount":amount, "type":'CREDIT', "description":"Payment from {}".format(self.number)})
        return self

In [14]:
help(Account)

Help on class Account in module __main__:

class Account(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, number, currency, balance=0)
 |      Bank account model
 |      
 |      Arguments
 |      ---------
 |      number   : the bank account number (e.g. IBAN)
 |      currency : the three-letter currency code of the account (e.g. USD) 
 |      balance  : the initial bank account balance
 |      
 |      Properties
 |      --------- 
 |      number       : the bank account number (e.g. IBAN)
 |      currency     : the three-letter currency code of the account (e.g. USD) 
 |      balance      : the initial bank account balance
 |      transactions : a list of transactions
 |  
 |  credits(self)
 |      Method to return a list of all the credit transactions made to the account
 |  
 |  debits(self)
 |      Method to return a list of all the debit transactions made to the account
 |  
 |  deposit(self, amount)
 |      Convenience method to add a new cash deposit
 |  
 | 

### 2.5 Enhancing encapsulation: read-only properties

In [16]:
#Create two accounts
checking = Account(number="#123456789", currency="USD", balance=1000)
savings = Account(number="#123456789", currency="USD", balance=0)

#Make a few payments
savings.deposit(1000)
savings.withdraw(500)
checking.send(savings, amount=100)

#Print the balances
print(savings.balance)
print(checking.balance)

#What happens if I directly override the balance property? 
savings.balance = 0 
print(savings.balance) #whoops - someone stole all my money! 

600
900
0


In [17]:
#We will tweak the Account class definition such that the .balance property is read-only
class Account:
    def __init__(self, number, currency, balance=0):
        #... same as above, but change
        #this is a convention: don't EVER touch underscored properties
        self._balance = balance
        
    def getbalance(self):
        return self._balance
    
    #here is the magic
    #you define balance as a read-only property using the property function
    #this function takes 1 to 4 arguments
    #the first one is the 'getter' function argument, named fget
    balance = property(fget=getbalance)

In [18]:
#Let's try it out
savings = Account("#123456789", "USD", balance=20)

#let's get the value
print(savings.balance)

#let's try to assign a new value
#this will throw an error, so let's catch it
try: 
    savings.balance = 100
except Exception as error: 
    print("Error:", error)

20
Error: can't set attribute


### 2.6. Read and write properties

In [19]:
class Account:
    def __init__(self, number, currency, balance=0):
        #this is a convention: don't EVER touch underscored properties
        self._category = "RETAIL" #one of "RETAIL", "PROFESSIONAL" or "INSTITUTIONAL"
        
    def getcategory(self):
        return self._category
    
    def setcategory(self, value):
        if value not in ["RETAIL","PROFESSIONAL","INSTITUTIONAL"]: 
            raise ValueError("Category must be one of RETAIL, PROFESSIONAL, INSTITUTIONAL")
        self._category = value
        
    category = property(getcategory, setcategory)

In [20]:
#let's try it out! 
#Let's try it out
savings = Account("#123456789", "USD", balance=20)
savings.category = 'RETAIL'

#let's get the value
print(savings.category)

#let's try to assign a new, invalid value
#this will throw an error, so let's catch it
try: 
    savings.category = 'UNKNWON'
except Exception as error: 
    print("Error:", error)

RETAIL
Error: Category must be one of RETAIL, PROFESSIONAL, INSTITUTIONAL
